In [57]:
using Gridap
using Gridap.Arrays

$$
    \begin{align*}
        - \nabla^2u &= 2 - (x^2 + y^2) \\
        u &= 0 \text{ on } \Gamma_1\\
        \frac{\partial u}{\partial n} &= 0 \text{ on } \Gamma_2
    \end{align*}
$$

In [51]:
domain = (0,1,0,1)
partition = (10,10)
model = CartesianDiscreteModel(domain,partition)


degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

GenericMeasure()

In [ ]:
labeling = get_face_labeling(model)
new_entity = num_entities(labeling) + 1
topo = get_grid_topology(model_fluid)
D = num_cell_dims(model_fluid)
for d in 0:D-1
    fluid_boundary_mask = collect(Bool,get_isboundary_face(topo,d))
    fluid_outer_boundary_mask = get_face_mask(labeling,"boundary",d)
    fluid_interface_mask = collect(Bool,fluid_boundary_mask .!= fluid_outer_boundary_mask)
    dface_list = findall(fluid_interface_mask)
    for face in dface_list
        labeling.d_to_dface_to_entity[d+1][face] = new_entity
    end
end
add_tag!(labeling,"interface",[new_entity])

In [39]:
function is_in(coords)
    V = [coord[1] == 0. || coord[2] == 0. for coord in coords]
    
end
oldcell_to_coods = get_cell_coordinates(Ω)
println(oldcell_to_coods)
a = lazy_map(is_in,oldcell_to_coods)
G1Mask = map(is_in, oldcell_to_coods)
G1Cells = findall(vcat(G1Mask...))

#vcat(G1Mask...)
#incell_to_cell = findall(oldcell_to_is_in)
#outcell_to_cell = findall(collect(Bool, .! oldcell_to_is_in))
#model_solid = DiscreteModel(model,incell_to_cell)
#model_fluid = DiscreteModel(model,outcell_to_cell)

# Triangulations
#trian_solid = Triangulation(model_solid)
#trian_fluid = Triangulation(model_fluid)

# Construct boundary triangulation at the fluid-solid interface
#topo = get_grid_topology(model_fluid)
#labeling = get_face_labeling(model_fluid)
#D = num_cell_dims(model_fluid)
#fluid_boundary_mask = collect(Bool,get_isboundary_face(topo,D-1))
#fluid_outer_boundary_mask = get_face_mask(labeling,"boundary",D-1)
#fluid_interface_mask = collect(Bool,fluid_boundary_mask .!= fluid_outer_boundary_mask)
#trian_Γi = BoundaryTriangulation(model_fluid,fluid_interface_mask)

Vector{VectorValue{2, Float64}}[[(0.0, 0.0), (0.5, 0.0), (0.0, 0.5), (0.5, 0.5)] [(0.0, 0.5), (0.5, 0.5), (0.0, 1.0), (0.5, 1.0)]; [(0.5, 0.0), (1.0, 0.0), (0.5, 0.5), (1.0, 0.5)] [(0.5, 0.5), (1.0, 0.5), (0.5, 1.0), (1.0, 1.0)]]
hi


LoadError: MethodError: no method matching DiscreteModel(::CartesianDiscreteModel{2, Float64, typeof(identity)}, ::Vector{Int64})

[0mClosest candidates are:
[0m  DiscreteModel(::Gridap.Geometry.Grid, [91m::Gridap.Geometry.GridTopology[39m, [91m::Gridap.Geometry.FaceLabeling[39m)
[0m[90m   @[39m [35mGridap[39m [90m~/.julia/packages/Gridap/I60jU/src/Geometry/[39m[90m[4mDiscreteModels.jl:438[24m[39m


In [3]:
writevtk(model,"testGrid1")

3-element Vector{Vector{String}}:
 ["testGrid1_0.vtu"]
 ["testGrid1_1.vtu"]
 ["testGrid1_2.vtu"]

In [52]:
f(x) = 2 - (x[1])
g(x) = 0

order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(model,reffe,conformity=:H1, dirichlet_tags="boundary")
U = TrialFESpace(V0, 0)

TrialFESpace()

In [53]:
a(u,v) = ∫( ∇(v) ⋅ ∇(u) ) * dΩ

b(v) = ∫( v * f ) * dΩ

b (generic function with 1 method)

In [54]:
op = AffineFEOperator(a,b,U,V0)

ls = LUSolver()
solver = LinearFESolver(ls)

uh = solve(op)

SingleFieldFEFunction():
 num_cells: 100
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 6050267299945715588

In [55]:
writevtk(Ω,"TestSol",cellfields=["uh"=>uh])

(["TestSol.vtu"],)

In [ ]:
?collect